<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=51eed1c4adbb039b24bc2347f20f097c2757603225dbbf7d0312674849c01c74
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-10 10:44:06
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.40 C
-------------------
Today PnL: 61.56 K (0.44%)
Current PnL: -28.44 L (-18.37%)
CY Booked + Current PnL: -14.15 L (-9.14%)
-------------------
Total profit:  1.22 L
Total loss:  -29.65 L
-------------------
Total Booked + Current PnL: 12.83 L (10.04%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.22%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 93.07 L (66.55%)
Deployed:  1.28 C
Current:  1.40 C
CAGR/XIRR %: 5.05%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,1.21,-17.03,20.64,0.09,17257.0,-17166.0,83611.0,94.02,40.0,M-SC,2.36,253.0,-0.99,0.62,9.85,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,3.33,-11.85,13.47,0.02,20727.0,-20687.0,153872.0,134.93,54.0,M-SC,16.67,234.0,-1.00,1.14,33.84,OX40N,NTT,PAINTS
43,ITC,452.00,0.60,-1.42,12.03,10.44,23734.0,-2851.0,197287.0,-41.15,47.0,X-LC,1.71,5.0,-0.12,1.46,4.06,X40,NTT,FMCG
50,MASFIN,398.61,0.63,-2.33,24.96,22.05,23887.0,-2280.0,95700.0,-15.99,55.0,H-SC,6.86,164.0,-0.10,0.71,39.11,XR,ATH,FINANCE
82,VOLTAS,1530.00,0.27,4.79,14.22,19.69,28572.0,9183.0,200925.0,-5.50,45.0,X-MC,2.56,78.0,0.32,1.49,11.91,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COFFEEDAY,80.0,4.88,-43.69,140.24,35.27,89664.0,-49613.0,63936.0,-57.75,33.0,L-SC,10.15,270.0,-0.55,0.47,56.48,XR,NTT,HOTELS
44,JCHAC,2282.0,3.89,-35.48,54.99,-0.01,45341.0,-45351.0,82454.0,16769.57,26.0,M-SC,4.13,233.0,-1.00,0.61,5.53,OX40N,NTT,AC
39,INDIGOPNTS,1408.0,3.33,-11.85,13.47,0.02,20727.0,-20687.0,153872.0,134.93,54.0,M-SC,16.67,234.0,-1.00,1.14,33.84,OX40N,NTT,PAINTS
49,LAOPALA,464.0,2.58,-38.55,126.34,39.09,101786.0,-50540.0,80565.0,76.43,32.0,H-SC,7.85,155.0,-0.50,0.60,3.76,AR,NTT,CERAMICS
30,HATHWAY,31.4,2.40,-29.25,144.93,73.29,111480.0,-31800.0,76920.0,794.93,42.0,H-SC,7.31,172.0,-0.29,0.57,3.72,XR,NTT,ENTERTAINMENT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HCLTECH,1922.01,-1.84,7.57,15.95,24.73,41509.0,18319.0,260243.0,12.34,62.0,X-LC,5.74,7.0,0.44,1.93,23.96,X40,ATH,IT
15,CAMS,4762.00,-1.40,-80.63,544.38,24.79,265516.0,-203082.0,48774.0,-81.54,37.0,X-SC,1.49,86.0,-0.76,0.36,19.23,X40N,NTT,MISC
61,ROUTE,2227.21,-1.39,-48.23,226.57,69.05,156066.0,-64184.0,68882.0,-58.05,48.0,H-SC,23.16,140.0,-0.41,0.51,6.23,SR,ATH,IT
23,DMART,5391.45,-1.13,-8.14,39.47,28.12,76292.0,-17117.0,193290.0,-20.72,33.0,X-LC,4.58,19.0,-0.22,1.43,13.57,X40N,ATH,FMCG
59,RELAXO,1176.00,-0.99,-46.03,182.18,52.31,142736.0,-66811.0,78349.0,-42.50,50.0,X-SC,6.10,91.0,-0.47,0.58,4.87,X40N,NTT,FOOTWEAR


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,1.71,-6.42,122.18,107.91,168810.0,-9479.0,138165.0,-23.79,40.0,M-SC,12.16,216.0,-0.06,1.02,3.02,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,3.33,-11.85,13.47,0.02,20727.0,-20687.0,153872.0,134.93,54.0,M-SC,16.67,234.0,-1.00,1.14,33.84,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,1.21,-17.03,20.64,0.09,17257.0,-17166.0,83611.0,94.02,40.0,M-SC,2.36,253.0,-0.99,0.62,9.85,OX40N,NTT,DURABLES
66,SIS,528.00,0.37,-24.38,60.36,21.26,50678.0,-27073.0,83959.0,1980.31,48.0,H-SC,4.70,166.0,-0.53,0.62,13.44,OX40N,NTT,MISC
47,KANSAINER,340.00,1.21,-25.04,51.38,13.47,103860.0,-67527.0,202140.0,-69.09,32.0,H-SC,3.65,158.0,-0.65,1.50,2.78,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.92,6.55,61.68,72.27,111357.0,11092.0,180540.0,-8.35,67.0,M-LC,2.07,99.0,0.1,1.34,13.13,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.92,6.55,61.68,72.27,111357.0,11092.0,180540.0,-8.35,67.0,M-LC,2.07,99.0,0.10,1.34,13.13,XR,NTT,IT
38,INDIAMART,4810.62,-0.32,-3.29,113.76,106.72,135688.0,-4060.0,119276.0,-52.15,32.0,H-SC,1.57,139.0,-0.03,0.88,19.32,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,1.52,-1.48,37.67,35.63,77777.0,-3105.0,206469.0,-15.41,45.0,H-MC,3.13,119.0,-0.04,1.53,14.94,AR,ATH,PHARMA
25,FINCABLES,1641.55,1.71,-6.42,122.18,107.91,168810.0,-9479.0,138165.0,-23.79,40.0,M-SC,12.16,216.0,-0.06,1.02,3.02,OX40N,ATH,CABLES
37,IEX,219.00,-0.16,-4.33,55.35,48.63,105882.0,-8658.0,191296.0,-36.24,47.0,H-SC,14.07,136.0,-0.08,1.42,6.72,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,DIXON,18931.72,0.44,-12.81,39.44,21.58,48193.0,-17946.0,122193.0,-56.03,26.0,X-MC,6.00,56.0,-0.37,0.91,8.18,X40N,ATH,IT
75,TMPV,600.00,0.12,-18.22,73.86,42.18,166954.0,-50369.0,226041.0,-26.74,26.0,X-LC,5.60,3.0,-0.30,1.68,0.12,XY24,NTT,AUTO
9,BAJAJHFL,181.50,1.12,-20.99,87.89,48.45,139239.0,-42082.0,158424.0,-26.75,27.0,X-MC,11.66,64.0,-0.30,1.17,3.15,X40N,ATH,FINANCE
33,HINDUNILVR,2922.00,-0.02,-10.29,26.71,13.67,61593.0,-26463.0,230600.0,-16.94,29.0,X-LC,2.79,14.0,-0.43,1.71,8.86,XY25,NTT,FMCG
2,ABBOTINDIA,35195.00,-0.78,-7.06,26.35,17.44,36699.0,-10572.0,139275.0,-20.03,30.0,X-MC,4.84,63.0,-0.29,1.03,10.75,X40,ATH,PHARMA


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.0,0.30,-21.89,96.52,53.50,227492.0,-66057.0,235694.0,-62.23,32.0,X-MC,1.34,58.0,-0.29,1.75,4.69,XY24,NTT,FMCG
15,CAMS,4762.0,-1.40,-80.63,544.38,24.79,265516.0,-203082.0,48774.0,-81.54,37.0,X-SC,1.49,86.0,-0.76,0.36,19.23,X40N,NTT,MISC
43,ITC,452.0,0.60,-1.42,12.03,10.44,23734.0,-2851.0,197287.0,-41.15,47.0,X-LC,1.71,5.0,-0.12,1.46,4.06,X40,NTT,FMCG
36,ICICIPRULI,790.0,1.25,4.67,25.12,30.97,46472.0,8262.0,185000.0,-20.00,59.0,X-MC,1.88,75.0,0.18,1.37,17.82,X40,ATH,INSURANCE
3,ACC,3906.0,0.22,-24.82,118.33,64.15,211692.0,-59051.0,178900.0,-55.88,37.0,X-SC,1.96,82.0,-0.28,1.33,0.35,XY24,BTT,CEMENT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,PGHH,17907.65,-0.02,-5.92,42.17,33.76,79676.0,-11880.0,188940.0,-33.58,34.0,X-MC,5.15,57.0,-0.15,1.40,0.07,X40,ATH,FMCG
75,TMPV,600.00,0.12,-18.22,73.86,42.18,166954.0,-50369.0,226041.0,-26.74,26.0,X-LC,5.60,3.0,-0.30,1.68,0.12,XY24,NTT,AUTO
53,PAGEIND,51605.08,0.20,-8.95,38.46,26.07,57336.0,-14660.0,149080.0,-32.90,31.0,X-MC,11.83,55.0,-0.26,1.11,0.20,X40,ATH,APPARELS
3,ACC,3906.00,0.22,-24.82,118.33,64.15,211692.0,-59051.0,178900.0,-55.88,37.0,X-SC,1.96,82.0,-0.28,1.33,0.35,XY24,BTT,CEMENT
31,HAVELLS,2069.16,-0.11,-9.19,45.89,32.48,136680.0,-30148.0,297843.0,-13.77,41.0,X-MC,5.24,67.0,-0.22,2.21,1.18,X40,ATH,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.24,-30.13,104.45,42.84,47439.0,-19588.0,45418.0,-54.27,40.0,X-SC,36.82,83.0,-0.41,0.34,2.75,XY24,NTT,MISC
15,CAMS,4762.00,-1.40,-80.63,544.38,24.79,265516.0,-203082.0,48774.0,-81.54,37.0,X-SC,1.49,86.0,-0.76,0.36,19.23,X40N,NTT,MISC
59,RELAXO,1176.00,-0.99,-46.03,182.18,52.31,142736.0,-66811.0,78349.0,-42.50,50.0,X-SC,6.10,91.0,-0.47,0.58,4.87,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.11,-36.44,112.73,35.20,92191.0,-46890.0,81780.0,7.29,40.0,X-SC,15.40,92.0,-0.51,0.61,3.88,X40,NTT,FOOTWEAR
22,DIXON,18931.72,0.44,-12.81,39.44,21.58,48193.0,-17946.0,122193.0,-56.03,26.0,X-MC,6.00,56.0,-0.37,0.91,8.18,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.96,-0.46,-13.22,37.46,19.28,112458.0,-45748.0,300208.0,-24.27,60.0,X-LC,5.49,1.0,-0.41,2.23,10.98,X40,ATH,IT
41,INFY,2275.00,-0.73,8.85,42.28,54.87,144000.0,27699.0,340587.0,-15.29,64.0,X-LC,3.38,2.0,0.19,2.52,17.87,X40,BTT,IT
75,TMPV,600.00,0.12,-18.22,73.86,42.18,166954.0,-50369.0,226041.0,-26.74,26.0,X-LC,5.60,3.0,-0.30,1.68,0.12,XY24,NTT,AUTO
81,VBL,671.64,0.85,-3.94,41.23,35.67,125092.0,-12441.0,303401.0,-15.30,55.0,X-LC,4.71,4.0,-0.10,2.25,9.37,X40N,ATH,FMCG
43,ITC,452.00,0.60,-1.42,12.03,10.44,23734.0,-2851.0,197287.0,-41.15,47.0,X-LC,1.71,5.0,-0.12,1.46,4.06,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-0.02,-37.68,113.23,32.89,54483.0,-29090.0,48117.0,-694.37,54.0,L-MC,5.74,259.0,-0.53,0.36,32.51,XR,NTT,BANKS
6,ASIANTILES,137.00,2.01,-17.93,121.36,81.67,93436.0,-16819.0,76991.0,6916.67,39.0,L-SC,14.28,271.0,-0.18,0.57,49.10,XR,NTT,CERAMICS
50,MASFIN,398.61,0.63,-2.33,24.96,22.05,23887.0,-2280.0,95700.0,-15.99,55.0,H-SC,6.86,164.0,-0.10,0.71,39.11,XR,ATH,FINANCE
13,BSOFT,831.70,-0.59,-22.43,94.39,50.79,102981.0,-31551.0,109102.0,-0.66,68.0,H-SC,5.84,151.0,-0.31,0.81,27.03,XR,ATH,IT
39,INDIGOPNTS,1408.00,3.33,-11.85,13.47,0.02,20727.0,-20687.0,153872.0,134.93,54.0,M-SC,16.67,234.0,-1.00,1.14,33.84,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,-0.59,-22.43,94.39,50.79,102981.0,-31551.0,109102.0,-0.66,68.0,H-SC,5.84,151.0,-0.31,0.81,27.03,XR,ATH,IT
84,WIPRO,420.00,0.92,6.55,61.68,72.27,111357.0,11092.0,180540.0,-8.35,67.0,M-LC,2.07,99.0,0.10,1.34,13.13,XR,NTT,IT
32,HCLTECH,1922.01,-1.84,7.57,15.95,24.73,41509.0,18319.0,260243.0,12.34,62.0,X-LC,5.74,7.0,0.44,1.93,23.96,X40,ATH,IT
56,RAJESHEXPO,518.00,1.21,-63.16,171.65,0.07,86740.0,-86644.0,50533.0,1752.27,56.0,L-SC,2.32,268.0,-1.00,0.37,25.68,OX40N,NTT,JEWELLERY
39,INDIGOPNTS,1408.00,3.33,-11.85,13.47,0.02,20727.0,-20687.0,153872.0,134.93,54.0,M-SC,16.67,234.0,-1.00,1.14,33.84,OX40N,NTT,PAINTS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.71
1,25,45.69
2,50,76.60


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.11
MC    30.32
LC    24.58
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.30
X40      22.43
X40N     10.89
XR       10.10
XY25      9.37
AR        9.34
OX40N     7.80
SR        1.03
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.52
X-MC    23.56
X-LC    21.02
M-SC    12.17
X-SC     7.01
H-MC     4.99
M-MC     1.41
L-SC     1.41
M-LC     1.34
H-LC     1.17
L-LC     1.05
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.37,-7.41,42.15
IT,12.78,-16.84,80.02
FINANCE,9.52,-20.66,71.64
ELECTRICAL,6.24,-11.61,52.49
PAINTS,6.13,-11.70,28.29
MISC,6.08,-58.28,123.12
INSURANCE,4.60,-1.69,37.53
PHARMA,4.11,-0.81,33.07
AUTO,3.41,-25.27,78.98


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3220531.0,21
AR,1312103.0,10
XR,1300716.0,13
X40,1013324.0,14
X40N,978945.0,9
OX40N,738214.0,10
XY25,378854.0,6
SR,280890.0,2
MH,83125.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3669338.0,25
M-SC,1416695.0,15
X-MC,1305336.0,16
X-SC,976324.0,8
X-LC,927716.0,11
H-MC,405216.0,3
L-SC,269840.0,3
M-LC,111357.0,1
H-LC,76392.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1285414.0      6
           AR         898005.0      5
M-SC       XY24       807671.0      6
H-SC       XR         777833.0      7
X-SC       X40N       504322.0      3
X-MC       X40        455339.0      7
           XY24       391679.0      3
X-LC       X40        378981.0      5
H-SC       OX40N      344071.0      4
M-SC       OX40N      307403.0      5
X-SC       XY24       292998.0      3
H-SC       SR         280890.0      2
X-MC       X40N       273239.0      4
X-LC       XY24       247581.0      2
H-MC       AR         210028.0      2
X-LC       X40N       201384.0      2
H-MC       XY24       195188.0      1
X-MC       XY25       185079.0      2
L-SC       XR         183100.0      2
X-SC       X40        179004.0      2
M-SC       XR         173943.0      2
           AR         127678.0      2
M-LC       XR         111357.0      1
X-LC       XY25        99770.0      2
L-SC       OX40N       86740.0      1
H-SC       MH          83125.0      1
H-LC       AR          76392.0      1
M-MC       XY25        55600.0      1
L-MC       XR          54483.0      1
L-LC       XY25        38405.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 54.0 seconds
